In [1]:
import anndata as ad
import pandas as pd
import shapely
import shapely.plotting as splot
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# all ccf plotting functions have been moved to ccf_plots.py in the ../code dir
import ccf_plots

%matplotlib inline

sns.set_style('white')

In [2]:
import warnings
warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

In [3]:
adata = ad.read_h5ad("../data/merfish_638850_AIT17.custom_CCF_annotated_TH_ZI_only_2023-05-04_00-00-00/atlas_brain_638850_AIT17_custom_CCF_annotated_TH_ZI_only.h5ad")
adata

In [4]:
# nice to have everything for plotting in the obs dataframe
adata.obs['cirro_x'] = adata.obsm['spatial_cirro'][:,0]
adata.obs['cirro_y'] = adata.obsm['spatial_cirro'][:,1]

In [5]:
ccf_th_names = ccf_plots.get_ccf_th_names()
print(f"{ccf_th_names = }")

# define and view CCF polygons

## calculate polygons

In [6]:
ccf_polygons = ccf_plots.get_ccf_polygons(adata.obs)

In [7]:
ccf_polygons_th_zi = {x: ccf_polygons[x] for x in ccf_th_names}

In [8]:
# polygon for whole thalamus outline
th_outline_polygons = ccf_plots.get_outline_polygon(adata.obs)
# th_outline_polygons

## view polygon overlays

In [9]:
ccf_plots.plot_ccf_overlay(adata.obs, ccf_polygons)

In [10]:
ccf_plots.plot_th_outline(th_outline_polygons, sections='1199651060', alpha=0.05)

### define thalamus subset (requires atlas download)

can skip now, result saved

In [11]:
# import urllib.request, json 
# with urllib.request.urlopen("http://api.brain-map.org/api/v2/structure_graph_download/1.json") as url:
#     atlas = json.load(url)['msg']

In [12]:
from bg_atlasapi.bg_atlas import BrainGlobeAtlas
atlas = BrainGlobeAtlas("allen_mouse_100um", brainglobe_dir='/scratch/brainglobe/')

In [13]:
ccf_polygons_th_zi = {x: ccf_polygons[x] for x in ccf_polygons.keys() 
                      #549: thalamus, 797: ZI
                      if (549 in atlas.structures[x]['structure_id_path'] 
                          and not 549==atlas.structures[x]['structure_id_path'][-1])
                      or 797 in atlas.structures[x]['structure_id_path']
                     }

In [14]:
len(ccf_polygons_th_zi)

## view thalamus subset with subclass labels

In [15]:
adata_neuronal = adata[adata.obs["division_id_label"].isin(["3 PAL-sAMY-TH-HY-MB-HB neuronal","2 Subpallium GABAergic","4 CBX-MOB-other neuronal"])].copy()
# cluster_id as string to simplify querying
adata_neuronal.obs['cluster_id'] = pd.Categorical(adata_neuronal.obs['cluster_id'].astype(int).astype(str))

In [16]:
# may need to tweak clustering params on fine 
ccf_plots.plot_ccf_overlay(adata.obs.loc[[],:], ccf_polygons_th_zi, sections=adata.obs['section'].unique(), highlight='all', 
                 bg_cells=adata_neuronal.obs, bg_shapes=True, min_group_count=0, legend=False)

In [17]:
ccf_plots.plot_ccf_overlay(adata_neuronal.obs, ccf_polygons_th_zi, highlight=ccf_th_names, point_hue='subclass_id_label', legend=False, outlines_only=True)

# Investigate prong 1 cluster-nucleus groups

In [18]:
nuclei_df = pd.read_csv("../resources/Prong 1 Vitessce links by nucleus.csv", index_col=0)

## VM

In [19]:
# obs = ccf_plots.get_obs_from_annotated_clusters('VM', adata, nuclei_df)
obs = adata_neuronal.obs.query("cluster_label=='1142 TH Prkcd Grin2c Glut_13'")
ccf_plots.plot_nucleus_cluster_comparison_slices(obs, ccf_polygons_th_zi, nuclei=['VM', 'VAL'], legend='both', bg_cells=adata_neuronal.obs, bg_shapes=False)

In [20]:
obs = adata_neuronal.obs.query("supertype_label=='TH Prkcd Grin2c Glut_13'")
obs['cluster_label'].value_counts().loc[lambda x: x>5]

In [21]:
# remove 1140? annotated as VPMpc
clusters = ['1138', '1141', '1142']
obs = adata.obs.loc[lambda df: df['cluster_label'].str[:4].isin(clusters)]
ccf_plots.plot_nucleus_cluster_comparison_slices(obs, ccf_polygons_th_zi, nuclei=['VM'], legend='cells', bg_cells=adata_neuronal.obs, bg_shapes=False, outlines_only=True)

### diff exp

In [22]:
import scanpy as sc

In [23]:
sc.tl.rank_genes_groups(adata_neuronal, 'cluster_id', groups=['1142'], reference='1138', method='wilcoxon', rankby_abs=True)
sc.pl.rank_genes_groups(adata_neuronal, n_genes=20)

In [24]:
adata_neuronal

In [25]:
section='1199651045'
gene = "Calb1"
nuclei=['VM', 'VAL']
highlight_nuclei=['VM']
ccf_plots.plot_expression_ccf(adata_neuronal, ccf_polygons_th_zi, section, gene, nuclei=nuclei, bg_shapes=False, 
                              highlight=highlight_nuclei, cmap='Blues', 
                              show_outline=True);

In [26]:
section='1199651045'
gene = "Necab1"
nuclei=['VM', 'VAL']
highlight_nuclei=['VM']
ccf_plots.plot_expression_ccf(adata_neuronal, ccf_polygons_th_zi, section, gene, nuclei=nuclei, 
                    bg_shapes=False, highlight=highlight_nuclei,
                    cmap='Blues', show_outline=True);

In [27]:
section='1199651045'
gene = "Spon1"
nuclei=['VM', 'VAL']
highlight_nuclei=['VM']
ccf_plots.plot_expression_ccf(adata_neuronal, ccf_polygons_th_zi, section, gene, nuclei=nuclei, 
                    bg_shapes=False, highlight=highlight_nuclei,
                    cmap='Blues', show_outline=True);

In [28]:
section='1199651048'
gene = "Cbln1"
nuclei=['VM', 'VAL']
highlight_nuclei=['VM']
ccf_plots.plot_expression_ccf(adata_neuronal, ccf_polygons_th_zi, section, gene, nuclei=nuclei, 
                    bg_shapes=False, highlight=highlight_nuclei,
                    cmap='Blues', show_outline=True);

In [29]:
section='1199651048'
gene = 'Tmie'
nuclei=['VM', 'VAL']
highlight_nuclei=['VM']
ccf_plots.plot_expression_ccf(adata_neuronal, ccf_polygons_th_zi, section, gene, nuclei=nuclei, 
                    bg_shapes=False, highlight=highlight_nuclei,
                    cmap='Blues', show_outline=True);

In [30]:
section='1199651045'
gene = 'Galnt18'
nuclei=['VM', 'VAL']
highlight_nuclei=['VM']
ccf_plots.plot_expression_ccf(adata_neuronal, ccf_polygons_th_zi, section, gene, nuclei=nuclei, 
                    bg_shapes=False, highlight=highlight_nuclei,
                    cmap='Blues', show_outline=True);

In [31]:
sc.tl.rank_genes_groups(adata_neuronal, 'supertype_label', groups=['TH Prkcd Grin2c Glut_13'], method='wilcoxon', tie_correct=True)
sc.pl.rank_genes_groups(adata_neuronal, n_genes=20)

In [32]:
sc.tl.rank_genes_groups(adata_neuronal, 'cluster_id', groups=['1142'], method='wilcoxon', tie_correct=True)
sc.pl.rank_genes_groups(adata_neuronal, n_genes=20)

In [33]:
# ISH diff ex search
# https://mouse.brain-map.org/gene/show/87708

section='1199651048'

gene = "Galnt18"
nuclei=['VM', 'VAL']
highlight_nuclei=['VM']
ccf_plots.plot_expression_ccf(adata_neuronal, ccf_polygons_th_zi, section, gene, nuclei=nuclei, 
                    bg_shapes=False, highlight=highlight_nuclei,
                    cmap='Blues', show_outline=True);

In [34]:
section='1199651057'
nuclei=['VM', 'VAL']
highlight_nuclei=['VM']
ccf_plots.plot_expression_ccf(adata_neuronal, ccf_polygons_th_zi, section, gene, nuclei=nuclei, 
                    bg_shapes=False, highlight=highlight_nuclei,
                    cmap='Blues', show_outline=True);

In [35]:
gene = "Stxbp6"
nuclei=['VM', 'VAL']
highlight_nuclei=['VM']
ccf_plots.plot_expression_ccf(adata_neuronal, ccf_polygons_th_zi, section, gene, nuclei=nuclei, 
                    bg_shapes=False, highlight=highlight_nuclei,
                    cmap='Blues', show_outline=True);

In [36]:
section='1199651048'

gene = "Spon1"
nuclei=['VM', 'VAL']
highlight_nuclei=['VM']
ccf_plots.plot_expression_ccf(adata_neuronal, ccf_polygons_th_zi, section, gene, nuclei=nuclei, 
                    bg_shapes=False, highlight=highlight_nuclei,
                    cmap='Blues', show_outline=True);

In [37]:
section='1199651045'
nuclei=['VM', 'VAL']
highlight_nuclei=['VM']
ccf_plots.plot_expression_ccf(adata_neuronal, ccf_polygons_th_zi, section, gene, nuclei=nuclei, 
                    bg_shapes=False, highlight=highlight_nuclei,
                    cmap='Blues', show_outline=True);

### outlining VM / composition

In [38]:
import geopandas as gp
neuron_points = gp.GeoSeries.from_xy(*adata_neuronal.obsm['spatial_cirro'].T, index=adata_neuronal.obs_names)

In [39]:
import hdbscan

adata_vm = adata[adata.obs.query("section=='1199651048' & cluster_id==1142").index].copy()
df = adata_vm.obs
X = df[['cirro_x','cirro_y']].values
df['spatial_cluster'] = hdbscan.HDBSCAN(min_samples=10).fit_predict(X)
core_clusters = df['spatial_cluster'].value_counts().index[:2]
df['spatial_group'] = df['spatial_cluster'].apply(lambda x: 'core' if x in core_clusters else 'other')

In [40]:
df['spatial_group'].value_counts()

In [41]:
sns.scatterplot(data=df,  x='cirro_x', y='cirro_y', hue='spatial_cluster', s=4,  legend=True, palette='tab10')

In [42]:
nucleus = ccf_plots.get_polygon_from_obs(df.query("spatial_group=='core'"))
subset = neuron_points.geometry.intersects(nucleus)
adata_neuronal.obs['in_nucleus'] = subset.values
adata_nucleus = adata_neuronal[subset]

In [43]:
# sc.pl.embedding(adata_nucleus, "spatial_cirro", color="supertype_label")
sc.pl.embedding(adata_nucleus, "spatial_cirro", color="cluster_label", s=20)

In [44]:
adata_neuronal.obs.query("section=='1199651048' & supertype_label=='TH Prkcd Grin2c Glut_13'").groupby(['cluster_label'])['in_nucleus'].apply(lambda x: x.value_counts())

In [45]:
# need to include VAL types ??

In [46]:
sc.tl.rank_genes_groups(adata_vm, 'spatial_group', groups=['core'], reference='other', method='wilcoxon', rankby_abs=True)
sc.pl.rank_genes_groups(adata_vm, groups=['core'], n_genes=20)

## anterior

In [47]:
nuclei=['AD', 'AV', 'AMd', 'AMv']
section = '1199651060'
# ccf_plots.plot_ccf_section(ccf_polygons_th_zi, section, highlight=nuclei, )
ccf_plots.plot_ccf_overlay(adata.obs.loc[[],:], ccf_polygons_th_zi, sections=[section], highlight=nuclei, legend='both', bg_cells=adata_neuronal.obs, bg_shapes=True, min_group_count=0)

In [48]:
clusters = [str(x) for x in list(range(1169, 1171+1)) + list(range(1095, 1098+1))]
obs = adata.obs.loc[lambda df: df['cluster_label'].str[:4].isin(clusters)]

ccf_plots.plot_nucleus_cluster_comparison_slices(obs, ccf_polygons_th_zi, nuclei=nuclei, legend='cells', bg_cells=adata_neuronal.obs, bg_shapes=False, outlines_only=True)

In [49]:
# ccf_plots.plot_nucleus_cluster_comparison_slices('AD', adata, nuclei_df, ccf_polygons_th_zi)
obs = ccf_plots.get_obs_from_annotated_clusters('AD', adata, nuclei_df)
ccf_plots.plot_nucleus_cluster_comparison_slices(obs, ccf_polygons_th_zi, nuclei='AD')

In [50]:

# ccf_plots.plot_nucleus_cluster_comparison_slices('AD', adata, nuclei_df, ccf_polygons_th_zi)
obs = ccf_plots.get_obs_from_annotated_clusters('AD', adata, nuclei_df)
ccf_plots.plot_nucleus_cluster_comparison_slices(obs, ccf_polygons_th_zi, nuclei='AD')

### diff exp

#### AV

In [51]:
sc.tl.rank_genes_groups(adata_neuronal, 'subclass_label', groups=['AV Col27a1 Glut'], method='wilcoxon', rankby_abs=True)
sc.pl.rank_genes_groups(adata_neuronal, n_genes=20)

##### Col27a1

In [52]:
section='1199651060'

gene = "Col27a1"
ant_nuclei=['AD', 'AV', 'AMd', 'AMv']
highlight_nuclei=['AD', 'AV']
ccf_plots.plot_expression_ccf(adata_neuronal, ccf_polygons, section, gene, nuclei=ant_nuclei, bg_shapes=False, 
                    cmap='Blues', show_outline=True, highlight=highlight_nuclei);

##### Sorcs3
Mathew's plots in brain1 suggested high expression exclusively in the '1097 AV Col27a1 Glut cluster' in the AV

That does not appear to be the case in brain3...

In [53]:
section='1199651060'

# Mathew's plots in brain1 suggested high expression exclusively in the '1097 AV Col27a1 Glut cluster' in the AV
# That does not appear to be the case in brain3...
gene = "Sorcs3"
ant_nuclei=['AD', 'AV', 'AMd', 'AMv']
highlight_nuclei=['AV']
ccf_plots.plot_expression_ccf(adata_neuronal, ccf_polygons, section, gene, nuclei=ant_nuclei, bg_shapes=False, 
                    cmap='Blues', show_outline=True, highlight=highlight_nuclei);

#### AD

In [54]:
sc.tl.rank_genes_groups(adata_neuronal, 'subclass_label', groups=['AD Serpinb7 Glut'], method='wilcoxon', rankby_abs=True)
sc.pl.rank_genes_groups(adata_neuronal, n_genes=20)

In [55]:
section='1199651060'

gene = "Syndig1l"
ant_nuclei=['AD', 'AV', 'AMd', 'AMv']
highlight_nuclei=['AD']
ccf_plots.plot_expression_ccf(adata_neuronal, ccf_polygons, section, gene, nuclei=ant_nuclei, bg_shapes=False, 
                    cmap='Blues', show_outline=True, highlight=highlight_nuclei);

#### AM

In [56]:
sc.tl.rank_genes_groups(adata_neuronal, 'supertype_label', groups=['TH Prkcd Grin2c Glut_9'], method='wilcoxon', rankby_abs=True)
sc.pl.rank_genes_groups(adata_neuronal, n_genes=20)

In [57]:
section='1199651060'

gene = "Cbln1"
ant_nuclei=['AD', 'AV', 'AMd', 'AMv']
highlight_nuclei=['AMd', 'AMv']
ccf_plots.plot_expression_ccf(adata_neuronal, ccf_polygons, section, gene, nuclei=ant_nuclei, bg_shapes=False, 
                    cmap='Blues', show_outline=True, highlight=highlight_nuclei);

In [58]:
adata_neuronal.obs['anterior_type'] = (adata_neuronal.obs['supertype_label'].str.contains('Glut_9') |
                                       adata_neuronal.obs['supertype_label'].str.contains('AD') |
                                       adata_neuronal.obs['supertype_label'].str.contains('AV')).astype(str)

In [59]:
sc.tl.rank_genes_groups(adata_neuronal, 'anterior_type', method='wilcoxon', rankby_abs=True)
sc.pl.rank_genes_groups(adata_neuronal, n_genes=20)

In [60]:
section='1199651060'

gene = "C1ql3"
nuclei=['AD', 'AV', 'AMd', 'AMv']
ccf_plots.plot_expression_ccf(adata_neuronal, ccf_polygons_th_zi, section, gene, nuclei=nuclei, bg_shapes=False, 
                    cmap='Blues', show_outline=True);

In [61]:
section='1199651060'

gene = "Shox2"
nuclei=['AD', 'AV', 'AMd', 'AMv']
ccf_plots.plot_expression_ccf(adata_neuronal, ccf_polygons_th_zi, section, gene, nuclei=nuclei, bg_shapes=False, 
                    cmap='Blues', show_outline=True);

## MD

In [62]:
obs = ccf_plots.get_obs_from_annotated_clusters('MD', adata, nuclei_df)
ccf_plots.plot_nucleus_cluster_comparison_slices(obs, ccf_polygons_th_zi, 'MD', bg_shapes=False, bg_cells=adata_neuronal.obs)

### diff exp

In [63]:
sc.tl.rank_genes_groups(adata_neuronal, 'cluster_id', groups=['1133'], method='wilcoxon', rankby_abs=True)
sc.pl.rank_genes_groups(adata_neuronal, n_genes=20)

## other

In [64]:
obs = ccf_plots.get_obs_from_annotated_clusters('PT', adata, nuclei_df)
ccf_plots.plot_nucleus_cluster_comparison_slices(obs, ccf_polygons_th_zi, nuclei='PT', bg_cells=adata_neuronal.obs, bg_shapes=False)

In [65]:
obs = adata_neuronal.obs.query("supertype_label.str.contains('PVT-PT')", engine='python')
ccf_plots.plot_nucleus_cluster_comparison_slices(obs, ccf_polygons_th_zi, nuclei=['PT','PVT'], bg_cells=adata_neuronal.obs, bg_shapes=False, legend='both')

In [66]:
obs = ccf_plots.get_obs_from_annotated_clusters('PVT', adata, nuclei_df)
ccf_plots.plot_nucleus_cluster_comparison_slices(obs, ccf_polygons_th_zi, nuclei=['PT','PVT'], bg_cells=adata_neuronal.obs, bg_shapes=False, legend='both')

In [67]:
obs = ccf_plots.get_obs_from_annotated_clusters('LGd', adata, nuclei_df)
ccf_plots.plot_nucleus_cluster_comparison_slices(obs, ccf_polygons_th_zi, nuclei=['LGd-co', 'LGd-ip', 'LGd-sh'], bg_cells=adata_neuronal.obs, bg_shapes=False)

In [68]:
obs = ccf_plots.get_obs_from_annotated_clusters('SMT', adata, nuclei_df)
ccf_plots.plot_nucleus_cluster_comparison_slices(obs, ccf_polygons_th_zi, nuclei='SMT', bg_cells=adata_neuronal.obs, bg_shapes=False)

In [69]:
obs = ccf_plots.get_obs_from_annotated_clusters('LD', adata, nuclei_df)
ccf_plots.plot_nucleus_cluster_comparison_slices(obs, ccf_polygons_th_zi, nuclei='LD', bg_cells=adata_neuronal.obs, bg_shapes=False)

# OLD: clustering tests

In [70]:
ccf_plots.plot_ccf_overlay(adata.obs, ccf_polygons)

In [71]:
from collections import defaultdict
from shapely.ops import unary_union
from sklearn.cluster import k_means, OPTICS, DBSCAN

data = adata.obs.query("section=='1199651024'")

ccf_polygons = defaultdict(dict)  
for (name, section), df in data.groupby(['CCF_acronym', 'section']):
    X = df[['cirro_x','cirro_y']].values
    if X.shape[0] > 100:
        clusters = OPTICS(min_cluster_size=50, min_samples=10, n_jobs=2).fit_predict(X)
        results = [ccf_plots.poly_from_points(X[clusters==i, :]) for i in set(clusters) if not i==-1]
        ccf_polygons[name][section] = unary_union(results)
        

In [72]:
df = adata.obs.query("section=='1199651033' & CCF_acronym=='MB'").copy()
X = df[['cirro_x','cirro_y']].values
df['spatial_cluster'] = hdbscan.HDBSCAN(min_samples=10).fit_predict(X)
sns.scatterplot(data=df,  x='cirro_x', y='cirro_y', hue='spatial_cluster', s=2,  legend=True, palette='tab10')

In [73]:
df = adata.obs.query("section=='1199651033' & CCF_acronym=='MB'").copy()
X = df[['cirro_x','cirro_y']].values
df['spatial_cluster'] = OPTICS(min_samples=10, xi=0.01, max_eps=1000, predecessor_correction=False, n_jobs=4).fit_predict(X)
sns.scatterplot(data=df,  x='cirro_x', y='cirro_y', hue='spatial_cluster', s=2,  legend=True, palette='tab10')